In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go


cars = pd.read_csv('cars-dataset.csv')
countries = pd.read_csv('brands_countries.csv')

/var/folders/s8/ryzdv1sn1g15nt9h4886lz7m0000gn/T/ipykernel_33044/2643789517.py:9: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  cars = pd.read_csv('cars-dataset.csv')


In [2]:
# Join the two dataframes along rows
complete = cars.merge(countries, on='Company', how='left')

#Save the complete dataset to a new csv file
complete.to_csv('complete_cars.csv', index=False)

In [3]:
number_of_brands = countries['Country'].value_counts()

In [4]:
# Primeira Palette de Cores
custom_colors = ['#1865A5', '#76C1EF','#DE3F47','#950E3F']

# Segunda Palette de Cores com +2 cores
line_colors = ['#1865A5', '#76C1EF', '#f9d29f', '#EBA74C', '#DE3F47','#950E3F']

## Number of Companies per Country (Worldmap)

In [5]:
country_counts = countries['Country'].value_counts()
# Create a DataFrame from the counts
df_counts = pd.DataFrame({'Country': country_counts.index, 'Count': country_counts.values})

fig = px.choropleth(df_counts, 
                    locations='Country', 
                    locationmode='country names', 
                    color='Count', 
                    hover_name='Country', 
                    title='Number of Comapnies per country',
                    color_continuous_scale=custom_colors,
                    labels={'Count':'Number of car brands'},
                    projection='natural earth',
                    scope='world',
                    width=1200,
                    height=800)

fig.show()

In [6]:
#Apagar informação desnecessária da coluna 'Power(HP)' -> deixar só a Horse Power (HP)
cars['HP'] = cars['Power(HP)'].str.split(' ').str[0]
#cars.drop('Power(HP)', axis=1, inplace=True)
#cars['HP']

In [7]:
#Apagar informação desnecessária da coluna 'Top Speed' -> deixar só em km/h (ignorar mph)
cars['Top Speed'].fillna('', inplace=True)
cars['Top Speed km/h'] = cars['Top Speed'].str.extract(r'(\d+)\s+km/h').astype(float)
#cars.drop(columns='Top Speed', inplace=True)
#cars['Top Speed km/h']

In [8]:
#Guardar velocidade média num ficheiro novo
cars.groupby('Company')['Top Speed km/h'].mean().to_csv('mean_top_speed_by_brand.csv')

## Velocidade Média por Marca

In [9]:
# Calcula a velocidade média
mean_top_speed = cars.groupby('Company')['Top Speed km/h'].mean().reset_index()

# Remove NaNs
mean_top_speed = mean_top_speed.dropna(subset=['Top Speed km/h'])

# Plot the mean top speed by brand, excluding rows with NaN values
fig = px.bar(mean_top_speed[mean_top_speed['Company'].notna()], 
             x='Company', 
             y='Top Speed km/h', 
             title='Mean top speed by brand',
             color='Top Speed km/h',
             color_continuous_scale=custom_colors,
             width=1200,
             height=600)

# Update layout to adjust the order of x-axis categories
fig.update_layout(xaxis={'categoryorder':'total descending'})

fig.show()

In [10]:
#Filter the cars that were produced in the selected year and calculate the mean top speed per company and add the year selected

evolution_top_speed = pd.DataFrame()

for i in range(1899, 2025):
    year_selected = i
    #print(i)
    aux_evolution_top_speed = cars[cars['Production years'].str.contains(str(year_selected))].groupby('Company')['Top Speed km/h'].mean()
    # change name of column
    aux_evolution_top_speed = aux_evolution_top_speed.reset_index()
    aux_evolution_top_speed.columns = ['Company', 'Mean Top Speed km/h']
    # add year column

    aux_evolution_top_speed['Year'] = year_selected

    evolution_top_speed = pd.concat([evolution_top_speed, aux_evolution_top_speed])

## Velocidade Média por Marca por Ano

In [11]:
brands_filter = ['BMW', 'AUDI', 'MERCEDES BENZ']

filtered_evolution_top_speed = evolution_top_speed[evolution_top_speed['Company'].isin(brands_filter)]

fig = px.line(filtered_evolution_top_speed,
                x='Year',
                y='Mean Top Speed km/h',
                title='Mean top speed by brand',
                color='Company',
                color_discrete_sequence=line_colors,
                width=1200,
                height=900,
                )

fig.update_traces(line=dict(width=4))

fig.update_layout(xaxis_title='Year',
                  yaxis_title='Mean Top Speed km/h',
                  title='Mean top speed by brand',
                  xaxis=dict(range=[1920, 2024]),
                  xaxis_rangeslider_visible=True)

fig.show()

## HP Média por Marca por Ano

In [12]:
# Average power by brand by year

cars['HP'] = cars['HP'].astype(float)

evolution_power_per_brand = pd.DataFrame()

for i in range(1899, 2025):
    year_selected = i
    #print(i)
    aux_evolution_power_per_brand = pd.DataFrame()
    aux_evolution_power_per_brand = cars[cars['Production years'].str.contains(str(year_selected))].groupby('Company')['HP'].mean()
    # change name of column
    aux_evolution_power_per_brand = aux_evolution_power_per_brand.reset_index()
    aux_evolution_power_per_brand.columns = ['Company', 'Mean HP']
    # add year column

    aux_evolution_power_per_brand['Year'] = year_selected

    evolution_power_per_brand = pd.concat([evolution_power_per_brand, aux_evolution_power_per_brand])


#evolution_power_per_brand
#cars.groupby('Company')['Power(HP)'].mean()

In [13]:
brands = ['TOYOTA', 'BUGATTI', 'FERRARI', 'BMW', 'AUDI', 'PORSCHE']

filtered_evolution_power = evolution_power_per_brand[evolution_power_per_brand['Company'].isin(brands)]

fig = px.line(filtered_evolution_power,
                x='Year',
                y='Mean HP',
                title='Mean HP by brand',
                color='Company',
                width=1200,
                height=900,
                color_discrete_sequence=line_colors
                )

fig.update_traces(line=dict(width=4))

fig.update_layout(xaxis_title='Year',
                  yaxis_title='Mean HP',
                  title='Mean HP by brand',
                  xaxis=dict(range=[1920, 2024]),
                  xaxis_rangeslider_visible=True)

fig.show()

## HP vs Cilindrada 

In [14]:
#cars['Displacement'] = cars['Displacement'].str.extract(r'(\d+)').astype(float)

cars['Displacement']

#print(cars['Fuel'].value_counts())

#Filter the cars that have the selected fuel type

fuel_type = 'Gasoline'

brands_filter = ['BMW', 'AUDI', 'MERCEDES BENZ', 'TOYOTA', 'BUGATTI', 'FERRARI']

cars_filtered = cars[cars['Fuel'] == fuel_type]

cars_filtered = cars_filtered[cars_filtered['Company'].isin(brands_filter)]

fig = px.scatter(cars_filtered,
                x='Displacement',
                y='HP',
                title='Power vs Displacement',
                color='Company',
                color_discrete_sequence=line_colors,
                hover_name="Model",
                hover_data=['Company', 'Production years', 'Specification summary'],
                width=1200,
                height=900,
                )



fig.show()

## Word Cloud de Marcas (Em progresso...)

In [15]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from ipywidgets import interact, widgets

def plot_wordcloud(country):
    # Filter the data based on the selected country
    brands_country = complete[complete['Country'] == country]
    # Create a dictionary with the brand names and the number of models
    brand_models = dict(brands_country['Company'].value_counts())
    # Create the word cloud
    wordcloud = WordCloud(width=800, height=400, background_color='white', colormap='RdBu').generate_from_frequencies(brand_models)  # Change colormap here
    # Plot the word cloud
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear', aspect='auto')
    plt.axis('off')
    plt.show()

# Create a dropdown menu with the countries
countries_list = complete['Country'].unique()
interact(plot_wordcloud, country=widgets.Dropdown(options=countries_list, description='Country:'))

interactive(children=(Dropdown(description='Country:', options=('United Kingdom', 'Japan', 'Italy', 'France', …

<function __main__.plot_wordcloud(country)>

## Número de Modelos Diferentes por Marca

In [16]:
unique_models = complete.groupby('Company')['Model'].nunique().reset_index()

fig = px.bar(unique_models,
                x='Company',
                y='Model',
                title='Number of unique models by brand',
                color='Model',
                color_continuous_scale=custom_colors,
                width=1200,
                height=600)

fig.update_layout(xaxis={'categoryorder':'total descending'})

fig.show()

## Word Cloud de Modelos (Em progresso...)

In [17]:
#Wordcloud with the series of the car

def plot_wordcloud_series(brand):
    # Filter the data based on the selected brand
    brand_series = complete[complete['Company'] == brand]
    # Create a dictionary with the series names and the number of models
    series_models = dict(brand_series['Serie'].value_counts())
    # Create the word cloud
    wordcloud = WordCloud(width=1600, height=500, background_color='white', colormap='RdBu').generate_from_frequencies(series_models)  # Change colormap here
    # Plot the word cloud
    plt.figure(figsize=(15,5))
    plt.imshow(wordcloud, interpolation='bilinear', aspect='auto')
    plt.axis('off')
    plt.show()

# Create a dropdown menu with the brands
brands_list = complete['Company'].unique()
interact(plot_wordcloud_series, brand=widgets.Dropdown(options=brands_list, description='Brand:'))

interactive(children=(Dropdown(description='Brand:', options=('AC', 'ACURA', 'ALFA ROMEO', 'ALPINE', 'ARIEL', …

<function __main__.plot_wordcloud_series(brand)>

## Número de Séries Diferentes por Marca

In [18]:
#Count the number of unique Series by brand and save the production years and company to a new csv file
unique_series = complete.groupby('Company')['Serie'].nunique().reset_index()

unique_series.to_csv('unique_series_by_brand.csv', index=False)

#Plot the number of unique series by brand

fig = px.bar(unique_series,
                x='Company',
                y='Serie',
                title='Number of unique series by brand',
                color='Serie',
                color_continuous_scale=custom_colors,
                width=1200,
                height=600)

fig.update_layout(xaxis={'categoryorder':'total descending'})

fig.show()

Filtragem das top 10 marcas com mais séries

In [19]:
top_10_brands = unique_series.nlargest(10, 'Serie')
fig = px.bar(top_10_brands,
                x='Company',
                y='Serie',
                title='Top 10 brands with the most unique series',
                color='Serie',
                color_continuous_scale=custom_colors,
                width=1200,
                height=600)

fig.update_layout(xaxis={'categoryorder':'total descending'})

fig.show()

## Top 10 Fabricantes por Ano (Marcas que mais modelos diferentes produziram)

In [20]:
top_manufacturers = pd.DataFrame()

for i in range(1899, 2025):
    year_selected = i
    #print(i)
    aux_top = pd.DataFrame()
    aux_top = cars[cars['Production years'].str.contains(str(year_selected))].groupby('Company')['Serie'].nunique()
    # change name of column
    aux_top = aux_top.reset_index()
    aux_top.columns = ['Company', 'Serie']
    # add year column
    aux_top['Year'] = year_selected
    top_manufacturers = pd.concat([top_manufacturers, aux_top])

In [21]:
top_manufacturers = top_manufacturers[top_manufacturers['Company'].isin(top_10_brands['Company'])]


fig = px.bar(top_manufacturers,
             x='Serie',
             y='Company',
             title='Top 10 manufacturers with the most unique series',
             color='Serie',
             color_continuous_scale=custom_colors,
             width=1200,
             height=600,
             hover_name='Company',
             animation_frame='Year',
             animation_group='Company'
            )

fig.update_layout(yaxis={'categoryorder': 'total ascending'})  # Sort bars by the number of series

fig.show()


## Comparação de Carros Diferentes (5 Diferentes)

In [22]:
from ipywidgets import Dropdown, HBox, VBox

def update_specifications(change):
    brand1 = brand_dropdown1.value
    serie1_dropdown.options = list(cars['Serie'][cars['Company'] == brand1].unique())
    specification1_dropdown.options = list(cars['Specification summary'][(cars['Company'] == brand1) & (cars['Serie'] == serie1_dropdown.value)].unique())
    
    brand2 = brand_dropdown2.value
    serie2_dropdown.options = list(cars['Serie'][cars['Company'] == brand2].unique())
    specification2_dropdown.options = list(cars['Specification summary'][(cars['Company'] == brand2) & (cars['Serie'] == serie2_dropdown.value)].unique())
    
    brand3 = brand_dropdown3.value
    serie3_dropdown.options = list(cars['Serie'][cars['Company'] == brand3].unique())
    specification3_dropdown.options = list(cars['Specification summary'][(cars['Company'] == brand3) & (cars['Serie'] == serie3_dropdown.value)].unique())
    
    brand4 = brand_dropdown4.value
    serie4_dropdown.options = list(cars['Serie'][cars['Company'] == brand4].unique())
    specification4_dropdown.options = list(cars['Specification summary'][(cars['Company'] == brand4) & (cars['Serie'] == serie4_dropdown.value)].unique())
    
    brand5 = brand_dropdown5.value
    serie5_dropdown.options = list(cars['Serie'][cars['Company'] == brand5].unique())
    specification5_dropdown.options = list(cars['Specification summary'][(cars['Company'] == brand5) & (cars['Serie'] == serie5_dropdown.value)].unique())

# Dropdown widgets for 5 cars
brand_dropdown1 = Dropdown(options=list(cars['Company'].unique()), description='Car 1 Brand:')
serie1_dropdown = Dropdown(description='Car 1 Serie:')
specification1_dropdown = Dropdown(description='Car 1 Specification:')

brand_dropdown2 = Dropdown(options=list(cars['Company'].unique()), description='Car 2 Brand:')
serie2_dropdown = Dropdown(description='Car 2 Serie:')
specification2_dropdown = Dropdown(description='Car 2 Specification:')

brand_dropdown3 = Dropdown(options=list(cars['Company'].unique()), description='Car 3 Brand:')
serie3_dropdown = Dropdown(description='Car 3 Serie:')
specification3_dropdown = Dropdown(description='Car 3 Specification:')

brand_dropdown4 = Dropdown(options=list(cars['Company'].unique()), description='Car 4 Brand:')
serie4_dropdown = Dropdown(description='Car 4 Serie:')
specification4_dropdown = Dropdown(description='Car 4 Specification:')

brand_dropdown5 = Dropdown(options=list(cars['Company'].unique()), description='Car 5 Brand:')
serie5_dropdown = Dropdown(description='Car 5 Serie:')
specification5_dropdown = Dropdown(description='Car 5 Specification:')

# Observe changes in dropdowns to update options dynamically
brand_dropdown1.observe(update_specifications, names='value')
serie1_dropdown.observe(update_specifications, names='value')

brand_dropdown2.observe(update_specifications, names='value')
serie2_dropdown.observe(update_specifications, names='value')

brand_dropdown3.observe(update_specifications, names='value')
serie3_dropdown.observe(update_specifications, names='value')

brand_dropdown4.observe(update_specifications, names='value')
serie4_dropdown.observe(update_specifications, names='value')

brand_dropdown5.observe(update_specifications, names='value')
serie5_dropdown.observe(update_specifications, names='value')

# Display the dropdowns for 5 cars in one line
HBox([VBox([brand_dropdown1, serie1_dropdown, specification1_dropdown]),
      VBox([brand_dropdown2, serie2_dropdown, specification2_dropdown]),
      VBox([brand_dropdown3, serie3_dropdown, specification3_dropdown]),
      VBox([brand_dropdown4, serie4_dropdown, specification4_dropdown]),
      VBox([brand_dropdown5, serie5_dropdown, specification5_dropdown])])


In [27]:
print("Car 1 :", brand_dropdown1.value, serie1_dropdown.value, specification1_dropdown.value)
print("Car 2 :", brand_dropdown2.value, serie2_dropdown.value, specification2_dropdown.value)
print("Car 3 :", brand_dropdown3.value, serie3_dropdown.value, specification3_dropdown.value)
print("Car 4 :", brand_dropdown4.value, serie4_dropdown.value, specification4_dropdown.value)
print("Car 5 :", brand_dropdown5.value, serie5_dropdown.value, specification5_dropdown.value)

car1 = {'Company': brand_dropdown1.value, 'Serie': serie1_dropdown.value, 'Specification': specification1_dropdown.value}
car2 = {'Company': brand_dropdown2.value, 'Serie': serie2_dropdown.value, 'Specification': specification2_dropdown.value}
car3 = {'Company': brand_dropdown3.value, 'Serie': serie3_dropdown.value, 'Specification': specification3_dropdown.value}
car4 = {'Company': brand_dropdown4.value, 'Serie': serie4_dropdown.value, 'Specification': specification4_dropdown.value}
car5 = {'Company': brand_dropdown5.value, 'Serie': serie5_dropdown.value, 'Specification': specification5_dropdown.value}


Car 1 : ACURA RDX 2.0L VTEC 10AT AWD (272 HP)
Car 2 : AC Aceca 4.6L V8 5MT (326 HP)
Car 3 : BRISTOL 400 2.0 4MT (81 HP)
Car 4 : BUFORI MK I 1.6 4MT (75 HP)
Car 5 : AURUS Senat S600 4.4L V8 9AT (598 HP)


In [29]:
# add column with car brand + specification summary

#cars['Name'] = cars['Company'] + ' ' + cars['Specification summary']

#cars['Torque(Nm)'] = cars['Torque(Nm)'].str.extract(r'(\d+)').astype(float)

#cars['Acceleration 0-62 Mph (0-100 kph)'] = cars['Acceleration 0-62 Mph (0-100 kph)'].str.extract(r'(\d+\.\d+)').astype(float)

In [30]:
def plot_car_comparison(car1brand, car1serie, car1specification, car2brand, car2serie, car2specification, car3brand, car3serie, car3specification, car4brand, car4serie, car4specification, car5brand, car5serie, car5specification):
    
    cars_filtered = cars[(cars['Company'] == car1brand) & (cars['Serie'] == car1serie) & (cars['Specification summary'] == car1specification)]
    cars_filtered = pd.concat([cars_filtered, cars[(cars['Company'] == car2brand) & (cars['Serie'] == car2serie) & (cars['Specification summary'] == car2specification)]])
    cars_filtered = pd.concat([cars_filtered, cars[(cars['Company'] == car3brand) & (cars['Serie'] == car3serie) & (cars['Specification summary'] == car3specification)]])
    cars_filtered = pd.concat([cars_filtered, cars[(cars['Company'] == car4brand) & (cars['Serie'] == car4serie) & (cars['Specification summary'] == car4specification)]])
    cars_filtered = pd.concat([cars_filtered, cars[(cars['Company'] == car5brand) & (cars['Serie'] == car5serie) & (cars['Specification summary'] == car5specification)]])
    
    figs = []

    fig = px.bar(cars_filtered,
                 x='HP',
                 y='Name',
                 title='Power comparison',
                 color='Company',
                 width=1200,
                 height=300,
                 )
    fig.show()
    fig = go.FigureWidget(fig)
    figs.append(fig)

    fig = px.bar(cars_filtered,
                 x='Torque(Nm)',
                 y='Name',
                 title='Torque comparison',
                 color='Company',
                 width=1200,
                 height=300,
                 )
    fig.show()

    fig = go.FigureWidget(fig)
    figs.append(fig)

    fig = px.bar(cars_filtered,
                 x='Acceleration 0-62 Mph (0-100 kph)',
                 y='Name',
                 title='Accelaration comparison',
                 color='Company',
                 width=1200,
                 height=300,
                 )
    fig.show()

    fig = go.FigureWidget(fig)
    figs.append(fig)


    HBox(figs)


In [31]:
plot_car_comparison(car1['Company'], car1['Serie'], car1['Specification'], car2['Company'], car2['Serie'], car2['Specification'], car3['Company'], car3['Serie'], car3['Specification'], car4['Company'], car4['Serie'], car4['Specification'], car5['Company'], car5['Serie'], car5['Specification'])